中文文档预处理及分词，可以输出为gensim能训练的文件格式，允许载入自定义词典，停用词词典，预处理包括繁简转换，去掉非中文字符，去掉停用词

jieba官方文档
https://github.com/fxsjy/jieba

简繁转换用opencc实现
参考http://blog.csdn.net/tab_space/article/details/50823073
`pip install opencc-python`

In [1]:
%%HTML
<button onclick="$('.output_stderr').toggle();">Toggle Code</button>

In [3]:
from jieba import posseg
import jieba
import re
import sys

#明确定义使用utf8解码，默认是ANSCII
reload(sys)  
sys.setdefaultencoding('utf8')

In [5]:
#一直有报错
#import opencc
#cc = opencc.OpenCC('t2s', data_path='/home/david/code/python/opencc/opencc-python-0.1/opencc/data')
#print cc.convert(u'Open Chinese Convert（OpenCC）「開放中文轉換」，是一個致力於中文簡繁轉換的項目，提供高質量詞庫和函數庫(libopencc)。')

In [6]:
'''
使用结巴进行分词，可以加载自定义词典和停用词词典，可以过滤非中文字符
segMode :
e -- 精确模式，试图将句子最精确地切开，适合文本分析，也是默认的模式
a -- 全模式，把句子中所有的可以成词的词语都扫描出来, 速度非常快，但是不能解决歧义
s -- 搜索引擎模式，在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词
userDictPath：用户自定义词典
stopWordPath：停用词词典
isFilterNoChinese：是否过滤非中文字符，默认过滤
'''
class WordSeg(object):
    def __init__(self, segMode, userDictPath = None, stopWordPath = None, isFilterNoChinese = True):
        self._segMode = segMode
        self._userDictPath = userDictPath
        self._stopWordPath = stopWordPath
        self._isFilterNoChinese = isFilterNoChinese
        
    def filterNoChinese(self, segStr):
        pattern = re.compile(ur'[^\u4e00-\u9fa5]')
        #去掉多余的空格 
        return re.sub(r"\s{2,}", "", " ".join(pattern.split(segStr)).strip())
    
    def generateStopWords(self):
        stopWords = {}
        if not self._stopWordPath == None:
            fstop = open(self._stopWordPath, 'r')
            for word in fstop:
                #解码为unicode进行处理
                stopWords[word.strip().decode('utf-8', 'ignore')] = word.strip().decode('utf-8', 'ignore')
        return stopWords
        
    def seg(self, segStr):
        segStr = segStr.strip().decode('utf8', 'ignore')
        if not self._userDictPath == None:
            jieba.load_userdict(self._userDictPath)
            
        if self._isFilterNoChinese:
            segStr = self.filterNoChinese(segStr)
        
        #多线程分词
        jieba.enable_parallel(5)
        wordGenList = []
        if self._segMode == 'a':
            wordGenList = [word for word in jieba.cut(segStr, cut_all=True) if word not in self.generateStopWords()]
        elif self._segMode == 's':
            wordGenList = [word for word in jieba.cut_for_search(segStr) if word not in self.generateStopWords()]
        else:
            wordGenList = [word for word in jieba.cut(segStr, cut_all=False) if word not in self.generateStopWords()]
        
        return wordGenList
    
    def posSeg(self, segStr):
        segStr = segStr.strip().decode('utf8', 'ignore')
        if not self._userDictPath == None:
            jieba.load_userdict(self._userDictPath)
            
        if self._isFilterNoChinese:
            segStr = self.filterNoChinese(segStr)
        wordPosGenList = posseg.cut(segStr, HMM=True)
        return wordPosGenList